# MDI Project summary

The [EVS 2017 integrated dataset (ZA7500)](https://europeanvaluesstudy.eu/methodology-data-documentation/survey-2017/full-release-evs2017/) is available as .sav (SPSS) and .dta (Stata) files under [DOI 10.4232/1.13560](https://doi.org/10.4232/1.13560). 

According to the authors, it contains data from the following 34 countries:

"Albania (AL); Armenia (AM); Austria (AT); Azerbaijan (AZ); Bosnia and Herzegovina (BA); Bulgaria (BG); Belarus (BY); Switzerland (CH); Czechia (CZ); Germany (DE); Denmark (DK); Estonia (EE); Spain (ES); Finland (FI); France (FR); Great Britain (GB); Georgia (GE); Croatia (HR); Hungary (HU); Iceland (IS); Italy (IT); Lithuania (LT); Montenegro (ME); Netherlands (NL); North Macedonia (MK); Norway (NO); Poland (PL); Portugal (PT); Romania (RO); Serbia (RS); Russia (RU); Sweden (SE); Slovenia (SI); Slovakia (SK)".

We first tried converting the SPSS file to .csv in SPSS 28. 
This would result in system crashes on a computer with 16 GB RAM.
Using the open source alternative [GNU PSPP](http://www.gnu.org/software/pspp/), we ran into the same issue.

Because of these issues, we settled on using Stata SE 16.1. 
In Stata, we opened the dataset `ZA7500-v4.0.0.dta` which we patched using `ZA7500-v4.0.0.Stata_PATCH_1.zip`. The patched dataset was then exported to a .csv file.

The notebook starts with importing this complete EVS dataset file. 


## Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl

from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE

from sklearn.metrics import silhouette_score
from sklearn.neighbors import DistanceMetric

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.spatial import distance_matrix, distance
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist

import networkx as nx
import numpy as np
import string
import graphviz

import itertools
import math

import json

import time
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from datetime import datetime as dt


## Filtering out irrelevant variables and reencoding answers into numbers

In [ ]:
# 500 MB upload with 30 Mbit/s, this takes around 2-3 minutes to run
data = pd.read_csv('https://cloud.hollander.online/s/Z8D9HbWJWJMP2Hf/download')

In [ ]:
# Print the column names + amount of columns
count_columns = 0
for col_name in data.columns: 
    print(col_name)
    count_columns = count_columns + 1


In [ ]:
print(count_columns)

Based on the [Survey published in Great Britain](https://dbk.gesis.org/dbksearch/download.asp?id=66252), also available through [DOI 10.4232/1.13560](https://doi.org/10.4232/1.13560), we decide that variables 'v6', 'v9', 'v36', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v93', 'v115', 'v134' and 'v196' are relevant to our research.

In [ ]:
# Filtering out the relevant variables from the original dataset
data_filtered = data.filter(['v6', 'v9', 'v36', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v93', 'v115', 'v134', 'v196'])

In [ ]:
# Counting the remaining number of columns
count = 0
for col_name in data_filtered.columns: 
    count = count + 1
print(count)

In [ ]:
# Show the data subset we just generated
data_filtered

### Reencoding the textual answers into numbers

We used `data_filtered["v6"].unique` to determine the answer options for each individual variable.
Based on these options, we reencoded answers as follows.

In [ ]:

data_filtered["v6"].replace({'not at all important': 1, 'not important': 2, 'quite important': 3, 'very important': 4, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v36"].replace({'trust completely': 1, 'trust somewhat': 2, 'do not trust very much': 3, 'do not trust at all': 4, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v54"].replace({'more than once week': 1, 'once a week': 2, "once a month": 3, "only on specific holy days": 4, "once a year": 5, "less often": 6, "never, practically never": 7, 'dont know':8, 'no answer':9, 'multiple answers Mail': 101}, inplace=True)
data_filtered["v55"].replace({'more than once week': 1, 'once a week': 2, "once a month": 3, "only on specific holy days": 4, "once a year": 5, "less often": 6, "never, practically never": 7, 'dont know':8, 'no answer':9, 'multiple answers Mail': 101}, inplace=True)
data_filtered["v56"].replace({'a religious person': 1, 'not a religious person': 2, "a convinced atheist": 3, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v63"].replace({'not at all important': 1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:8, 9:9, "very important": 10, 'dont know':88, 'no answer':99}, inplace=True)
data_filtered["v9"].replace({'mentioned': 1, 'not mentioned': 2, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v51"].replace({'yes': 1, 'no': 2, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v52"].replace({'Muslim': 1, 'Orthodox': 2, 'not applicable': 77, 'Roman catholic': 3,
       'Protestant': 4, 'Other': 5, 'no answer': 99, 'dont know': 88, 'Jew': 6, 'Buddhist': 7,
       'Free church/Non-conformist/Evangelical': 8, 'Hindu': 9}, inplace=True)
data_filtered["v53"].replace({'yes': 1, 'no': 2, 'dont know':8, 'no answer':9, 'not applicable': 7}, inplace=True)
data_filtered["v57"].replace({'yes': 1, 'no': 2, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v58"].replace({'yes': 1, 'no': 2, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v59"].replace({'yes': 1, 'no': 2, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v60"].replace({'yes': 1, 'no': 2, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v61"].replace({'yes': 1, 'no': 2, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v62"].replace({'personal God': 1, 'spirit or life force': 2, "don't know what to think": 3, "no spirit, God or life force": 4, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v93"].replace({'not mentioned': 2, 'mentioned': 1, 'no answer': 9, 'dont know': 8}, inplace=True)
data_filtered["v64"].replace({'every day': 1, 'once a week': 3, 'never': 7, 'several times a year': 5, 'at least once a month': 4, 'more than once week': 2, 'less often': 6,'no answer': 9, 'dont know': 8, 'multiple answers Mail': 101, 'other missing': 100}, inplace=True)
data_filtered["v115"].replace({'a great deal': 1, 'quite a lot': 2, 'not very much': 3, 'none at all': 4, 'dont know':8, 'no answer':9}, inplace=True)
data_filtered["v134"].replace({'not at all an essential characteristic of democracy': 1, 'dont know': 88, 'no answer': 99, 'an essential characteristic of democracy': 10 ,'it is against democracy [DO NOT READ OUT]': 0, 'item not included': 100, 'multiple answers Mail': 101}, inplace=True)
data_filtered["v196"].replace({'not at all important': 1, 'not important': 2, 'quite important': 3, 'very important': 4, 'dont know':8, 'no answer':9}, inplace=True)


## Cleaning the dataset by dropping unusable responses

Rows containing the following variables were dropped

- 8: "dont know"
- 88: "dont know"
- 9: "no answer"
- 99: "no answer"
- 100: "item not included"
- 101: "multiple answers Mail"

In [ ]:
data_cleaning = data_filtered

In [ ]:
data_cleaning = data_cleaning[data_cleaning.v6 != 8]
data_cleaning = data_cleaning[data_cleaning.v6 != 9]
data_cleaning = data_cleaning[data_cleaning.v9 != 8]
data_cleaning = data_cleaning[data_cleaning.v9 != 9]
data_cleaning = data_cleaning[data_cleaning.v36 != 8]
data_cleaning = data_cleaning[data_cleaning.v36 != 9]
data_cleaning = data_cleaning[data_cleaning.v51 != 8]
data_cleaning = data_cleaning[data_cleaning.v51 != 9]
data_cleaning = data_cleaning[data_cleaning.v52 != 88]
data_cleaning = data_cleaning[data_cleaning.v52 != 99]
data_cleaning = data_cleaning[data_cleaning.v53 != 8]
data_cleaning = data_cleaning[data_cleaning.v53 != 9]
data_cleaning = data_cleaning[data_cleaning.v54 != 8]
data_cleaning = data_cleaning[data_cleaning.v54 != 9]
data_cleaning = data_cleaning[data_cleaning.v54 != 101]

data_cleaning = data_cleaning[data_cleaning.v55 != 8]
data_cleaning = data_cleaning[data_cleaning.v55 != 9]
data_cleaning = data_cleaning[data_cleaning.v56 != 8]
data_cleaning = data_cleaning[data_cleaning.v56 != 9]
data_cleaning = data_cleaning[data_cleaning.v57 != 8]
data_cleaning = data_cleaning[data_cleaning.v57 != 9]
data_cleaning = data_cleaning[data_cleaning.v58 != 8]
data_cleaning = data_cleaning[data_cleaning.v58 != 9]
data_cleaning = data_cleaning[data_cleaning.v59 != 8]
data_cleaning = data_cleaning[data_cleaning.v59 != 9]
data_cleaning = data_cleaning[data_cleaning.v60 != 8]
data_cleaning = data_cleaning[data_cleaning.v60 != 9]
data_cleaning = data_cleaning[data_cleaning.v61 != 8]
data_cleaning = data_cleaning[data_cleaning.v61 != 9]

data_cleaning = data_cleaning[data_cleaning.v62 != 8]
data_cleaning = data_cleaning[data_cleaning.v62 != 9]
data_cleaning = data_cleaning[data_cleaning.v63 != 88]
data_cleaning = data_cleaning[data_cleaning.v63 != 99]
data_cleaning = data_cleaning[data_cleaning.v64 != 8]
data_cleaning = data_cleaning[data_cleaning.v64 != 9]
data_cleaning = data_cleaning[data_cleaning.v64 != 100]
data_cleaning = data_cleaning[data_cleaning.v93 != 8]
data_cleaning = data_cleaning[data_cleaning.v93 != 9]
data_cleaning = data_cleaning[data_cleaning.v115 != 8]
data_cleaning = data_cleaning[data_cleaning.v115 != 9]
data_cleaning = data_cleaning[data_cleaning.v134 != 88]
data_cleaning = data_cleaning[data_cleaning.v134 != 99]
data_cleaning = data_cleaning[data_cleaning.v134 != 100]
data_cleaning = data_cleaning[data_cleaning.v196 != 8]
data_cleaning = data_cleaning[data_cleaning.v196 != 9]

After performing the cleanup, we verify our work by checking which unique answers are left.

In [ ]:
varlist = []
for col_name in data_cleaning.columns: 
    varlist.append(col_name)
for var in varlist:
  print(var, data_cleaning[var].unique())

# Normalising the answers 

The scale answers differ from 1-4 to 1-7. They are normalised to a value between 0-1

Boolean variables are encoded as 1 or 2. They are reencoded as 0 or 1.

In [ ]:
data_normalised = data_cleaning

# Vars that need to be normalized (0-1): v6, v36, v54, v55, v63, v115, v134, v196
# Vars that are booleans have value 1 or 2, should be 0 or 1: v9, v51, v57, v58, v59, v60, v61, v93

# A subset is created containing all variables which need to be modified
normalise_subset = data_normalised[["v6", "v9", "v36", "v51", "v53", "v54", "v55", "v57", "v58", "v59", "v60", "v61", "v63", "v64", "v93", "v115", "v134", "v196"]]

# The subset is normalised
normalise_subset = (normalise_subset-normalise_subset.min())/(normalise_subset.max()-normalise_subset.min())

# And replaces the original data
data_normalised.update(normalise_subset)
# data_normalised.reset_index(inplace=True)

data_normalised.head(20)

In [ ]:
# Automatically convert variable types

data_normalised = data_normalised.convert_dtypes()

# print(data.dtypes)

# Set booleans
data_normalised['v9'] = data_normalised['v9'].astype('bool')
data_normalised['v51'] = data_normalised['v51'].astype('bool')
data_normalised['v57'] = data_normalised['v57'].astype('bool')
data_normalised['v58'] = data_normalised['v58'].astype('bool')
data_normalised['v59'] = data_normalised['v59'].astype('bool')
data_normalised['v60'] = data_normalised['v60'].astype('bool')
data_normalised['v61'] = data_normalised['v61'].astype('bool')
data_normalised['v93'] = data_normalised['v93'].astype('bool')

# Set categorical
data_normalised['v52'] = data_normalised['v52'].astype('category')
data_normalised['v56'] = data_normalised['v56'].astype('category')
data_normalised['v62'] = data_normalised['v62'].astype('category')

print(data_normalised.dtypes)

## Dropping duplicates

During the project, we decided to drop part of the data. However, we later abandoned this when we moved on to use a different approach.

In [ ]:
data_deduplicated = data_normalised
data_deduplicated = data_deduplicated.groupby(data_deduplicated.columns.tolist()).apply(len)\
      .rename('group_count')\
      .reset_index()
#      .sort_values(['group_count'], ascending = False)
data_deduplicated

In [ ]:
# Checking the resulting group counts
data_deduplicated['group_count'].unique()


In [ ]:
# Checking all columns are included
data_deduplicated.columns.tolist()

In [ ]:
# Check if type settings are still in order.
print(data_deduplicated.dtypes)

## Clearing RAM

To delay running out of RAM, we drop some data

In [ ]:
data = 0
data_filtered = 0
data_cleaning = 0

## Clustering



In [ ]:
data_clustered = data_normalised

# Empty data_normalised
data_normalised = 0


### PCA (For fun and to get familiar with clustering)

In [ ]:
pca = PCA(n_components='mle', svd_solver='full')
pca.fit(data_clustered)
PCA(n_components=2, svd_solver='full')
print(pca.explained_variance_ratio_)
# print(pca.singular_values_)

### Trying t-SNE to reduce dataset to 2D

In [ ]:
#m = TSNE(learning_rate=10)
m = TSNE(learning_rate=1500, random_state=234, perplexity=125, verbose=1)

# To save time, we can limit the amount of rows to use when experimenting.
# data_clustered = data_clustered.sample(n=250).drop(['group_count'], axis=1)
# data_clustered.head

tsne_features = m.fit_transform(data_clustered)
tsne_features[1:4,:]

In [ ]:
# Add the resulting t-SNE feature as columns to the original dataset.
data_clustered['x'] = tsne_features[:,0]
data_clustered['y'] = tsne_features[:,1]

In [ ]:
# Plot the t-SNE points using religion as colour encoding
sns.scatterplot(x="x", y="y", hue="v52", data=data_clustered)

In [ ]:
coords = data_clustered[['y', 'x']].to_numpy() 
score = silhouette_score(coords, data_clustered['v52'], metric='euclidean')
print('Silhouette Score based on religion: %.3f' % score)


### Trying K-Means on t-SNE data

In [ ]:
data_subset_km = data_clustered[['x', 'y']]

sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(data_subset_km)
    sum_of_squared_distances.append(km.inertia_)

plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
km = KMeans(n_clusters=4)

km.fit(data_subset_km)

data_subset_km['cl'] = km.labels_
data_subset_km.plot.scatter('x', 'y', c='cl', colormap='gist_rainbow')

### Calculating distance matrix

In [ ]:
dist = DistanceMetric.get_metric('euclidean')

two_d = data_clustered[['x', 'y']].to_numpy()

distance = dist.pairwise(data_clustered[['x', 'y']].to_numpy())

# This command will often crash the runtime environment because of lack of RAM when not using a reduced data set
#dm = distance_matrix(two_d_array['x'], two_d_array['y'])

This code calculates the Euclidean distance for each possible pair of nodes. The input for this code are the 2D coordinates of the dataset that was reduced in dimensionality with t-SNE. This data must be converted to a 2D array with the .to_numpy() function.

In [ ]:
# The following code will make the Google Colab runtime crash by using all available RAM when not using a reduced data set

# pairwise = pd.DataFrame(
#     squareform(pdist(data_clustered[['x', 'y']].to_numpy())),
#     columns = data_clustered[['x', 'y']].index,
#     index = data_clustered[['x', 'y']].index
# )

OpMAP requires us to remove edges with low similarity, therefore:
If distance < 10, set to NaN

In [ ]:
pairwise.applymap(lambda x: np.nan if x > 10 else x)

Plot the distance matrix

In [ ]:
G = nx.from_pandas_adjacency(pairwise)
G = nx.relabel_nodes(G, dict(zip(range(len(G.nodes())),string.ascii_uppercase)))    

G = nx.drawing.nx_agraph.to_agraph(G)

G.node_attr.update(color="red", style="filled")
G.edge_attr.update(color="blue", width="0.1")

G.draw('/tmp/out.png', format='png', prog='neato')

pos = nx.spring_layout(G)

# smaller nodes and fonts
plt.figure(2)
nx.draw(G,pos,node_size=60,font_size=8) 
# larger figure size
plt.figure(3,figsize=(12,12)) 
nx.draw(G,pos)
plt.show()

Export the edges to json, so they can be read by OpMAP

In [ ]:
pairwiseJSON = pairwise.to_json()

In [ ]:
with open('personal.json', 'w') as json_file:
    json.dump(pairwiseJSON, json_file)

### DBSCAN to reduce the number of nodes

The code in this section is adapted from <https://arxiv.org/abs/1803.08101>

In [ ]:
# command to display matplotlib plots inline within the ipython notebook
%matplotlib inline

In [ ]:
dataset_dbscan = data_clustered[['x', 'y']]

clustering = DBSCAN(eps=5, min_samples=150).fit(dataset_dbscan)

In [ ]:
sns.scatterplot(x="x", y="y", hue=clustering.labels_, palette="tab10", data=dataset_dbscan)

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
coords = dataset_dbscan
score = silhouette_score(coords, clustering.labels_, metric='euclidean')
print('Silhouette Score based on religion: %.3f' % score)

In [ ]:
def get_centermost_point(cluster):
    centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
    centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
    return tuple(centermost_point)

In [ ]:
def dbscan_reduce(df, epsilon, x='x', y='y'):
    start_time = time.time()
    coords = df[[y, x]].to_numpy()    
    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='auto', metric='euclidean').fit(coords)
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    print('\nEpsilon: {:,}'.format(epsilon))
    print('Number of clusters: {:,}'.format(num_clusters))
    
    clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
    
    # find the point in each cluster that is closest to its centroid
    centermost_points = clusters.map(get_centermost_point)

    # unzip the list of centermost points (lat, lon) tuples into separate lat and lon lists
    lats, lons = zip(*centermost_points)
    rep_points = pd.DataFrame({x:lons, y:lats})
    rep_points.tail()
    
    # pull row from original data set where lat/lon match the lat/lon of each row of representative points
    rs = rep_points.apply(lambda row: df[(df[y]==row[y]) & (df[x]==row[x])].iloc[0], axis=1)
    
    # all done, print outcome
    message = 'Clustered {:,} points down to {:,} points, for {:.2f}% compression in {:,.2f} seconds.'
    print(message.format(len(df), len(rs), 100*(1 - float(len(rs)) / len(df)), time.time()-start_time))

    score = silhouette_score(coords, db.labels_, metric='euclidean')
    print('Silhouette Score: %.3f' % score)

    return rs

In [ ]:
# first cluster the full data set
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.25)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.4)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.5)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.55)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.6)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.65)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.7)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.71)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.72)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.73)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.74)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.75)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.8)
data_reduced = dbscan_reduce(data_clustered, epsilon=0.81)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.82)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.83)

# data_reduced = dbscan_reduce(data_clustered, epsilon=0.85)
# data_reduced = dbscan_reduce(data_clustered, epsilon=0.9)

# data_reduced = dbscan_reduce(data_clustered, epsilon=1)
# data_reduced = dbscan_reduce(data_clustered, epsilon=2)


In [ ]:
data_reduced

In [ ]:
sns.scatterplot(x="x", y="y", hue="v52", data=data_reduced)

In [ ]:
# next, sample every nth row (where n=sample_rate) of the full data set
sample_rate = 20
df_sampled = data_clustered.iloc[range(0, len(data_clustered), sample_rate)]
len(df_sampled)

In [ ]:
# combine the clustered and sampled sets
df_combined = pd.concat([data_clustered, df_sampled], axis=0)
df_final = df_combined.reset_index().drop(labels='index', axis=1)

In [ ]:
# show a map of the worldwide data points
fig, ax = plt.subplots(figsize=[11, 8])
#rs_scatter = ax.scatter(df_final['x'], df_final['y'], c='m', edgecolor='None', alpha=0.3, s=120)
# rs_scatter = ax.scatter(data_clustered['x'], data_clustered['y'], c='m', edgecolor='None', alpha=0.3, s=120)
rs_scatter = ax.scatter(data_clustered['x'], data_clustered['y'], c=data_clustered['v52'], edgecolor='None', alpha=0.3, s=120)

df_scatter = ax.scatter(data_clustered['x'], data_clustered['y'], c='k', alpha=0.5, s=3)
ax.set_title('Full data set vs DBSCAN reduced set')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
#ax.legend([df_scatter, rs_scatter], ['Full set', 'Reduced set'], loc='upper right')

plt.show()

In [ ]:
# show a map of the worldwide data points
fig2, ax = plt.subplots(figsize=[11, 8])
#rs_scatter = ax.scatter(df_final['x'], df_final['y'], c='m', edgecolor='None', alpha=0.3, s=120)
# rs_scatter = ax.scatter(data_clustered['x'], data_clustered['y'], c='m', edgecolor='None', alpha=0.3, s=120)
rs_scatter = ax.scatter(data_clustered['x'], data_clustered['y'], c=data_clustered['v52'], edgecolor='None', alpha=0.3, s=120)

ax.set_title('DBSCAN reduced set')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
#ax.legend([df_scatter, rs_scatter], ['Full set', 'Reduced set'], loc='upper right')

plt.show()

In [ ]:
# Parameter Tuning for eps-value

from sklearn.neighbors import NearestNeighbors

# The larger the data set, the larger the value of MinPts should be.
# If the data set is noisier, choose a larger value of MinPts.
# Generally, MinPts should be greater than or equal to the dimensionality of the data set.

neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(data_subset)
distances, indices = nbrs.kneighbors(data_subset)

In [ ]:
# The optimal value for epsilon will be found at the point of maximum curvature.
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

In [ ]:
# show a map of the worldwide data points
fig, ax = plt.subplots(figsize=[11, 8])
rs_scatter = ax.scatter(data_clustered['x'], data_clustered['y'], c='m', edgecolor='None', alpha=0.3, s=120)
# df_scatter = ax.scatter(data_clustered['x'], data_clustered['y'], c='k', alpha=0.5, s=3)
ax.set_title('Full data set vs DBSCAN reduced set')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend([df_scatter, rs_scatter], ['Full set', 'Reduced set'], loc='upper right')
plt.show()

### K-Means optimal number of clusters on full data set

In [ ]:
df_km = data_clustered.drop(['x', 'y', 'id'], axis=1)

sum_of_squared_distances = []
K = range(1,15)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(df_km)
    sum_of_squared_distances.append(km.inertia_)

plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

In [ ]:
km = sk.cluster.KMeans(n_clusters=6)

km.fit(df_km)

df_km['cl'] = km.labels_

df_km['cl']
# df_km.plot.scatter('x', 'y', c='cl', colormap='gist_rainbow')

### Exporting the clustered data for D3.js

In [ ]:
data_clustered['id'] = np.arange(len(data_clustered))

data_clustered.head(5)

export_test = data_clustered.drop(['x', 'y'], axis=1).to_json(orient="records")

export_test

In [ ]:
with open('node_info.json', 'w') as f:
    json.dump(parsed, f)

In [ ]:
pairwise_dbscan = pd.DataFrame(
    squareform(pdist(data_clustered[['x', 'y']].to_numpy())),
    columns = data_clustered[['x', 'y']].index,
    index = data_clustered[['x', 'y']].index
)

In [ ]:
pairwise_dbscan_clean = pairwise_dbscan.applymap(lambda x: np.nan if x > 5 or x==0 else x)

pairwise_dbscan_clean

In [ ]:
# Scale values to [0, 10] but inverse (so distance becomes weight. High distance = low weight and vice versa)

pairwise_dbscan_clean_inv = pairwise_dbscan_clean.applymap(lambda x: 5 - x)

pairwise_dbscan_clean_inv

In [ ]:
import networkx as nx
import numpy as np
import string
# import pygraphviz

G = nx.from_pandas_adjacency(pairwise_dbscan_clean_inv)

# G = nx.relabel_nodes(G, dict(zip(range(len(G.nodes())),string.ascii_uppercase)))    

# G = nx.drawing.nx_agraph.to_agraph(G)

# G.node_attr.update(color="red", style="filled")
# G.edge_attr.update(color="blue", width="0.1")

# G.draw('/tmp/out.png', format='png', prog='neato')

pos = nx.spring_layout(G)

# smaller nodes and fonts
plt.figure(2)
nx.draw(G,pos,node_size=60,font_size=8) 
# larger figure size
plt.figure(3,figsize=(12,12)) 
nx.draw(G,pos)
plt.show()

In [ ]:
from networkx.readwrite import json_graph

data2 = json_graph.node_link_data(
    G, {"link": "edges", "source": "source", "target": "target", "weight": "value"}
)

In [ ]:
with open('nodes.json', 'w') as fp:
    fp.write(
        '[' +
        ',\n'.join(json.dumps(i) for i in data2['nodes']) +
        ']\n')

In [ ]:
data2['nodes']

In [ ]:
with open('edges.json', 'w') as fp:
    fp.write(
        '[' +
        ',\n'.join(json.dumps(i) for i in data2['edges']) +
        ']\n')

### Agglomerative clustering

In [ ]:
clustering = AgglomerativeClustering(affinity='precomputed', linkage='complete', n_clusters=6)

p = clustering.fit_predict(pairwise)

p.shape
# AgglomerativeClustering()
# clustering.labels_
# array([1, 1, 1, 0, 0, 0])

In [ ]:
clustering.labels_

In [ ]:
def get_dist(point_1:tuple, point_2:tuple):
    return math.sqrt((point_1[0]-point_2[0])**2 + (point_1[1]-point_2[1])**2)

def process(input_list, threshold=10):   
    combos = itertools.combinations(input_list, 2)
    points_to_remove = [point2 for point1, point2 in combos if get_dist(point1, point2)<=threshold]
    p_t_r = np.vstack(points_to_remove)
    # print(len(p_t_r))
    # print(len(input_list))
    # print(input_list)

    points_to_keep = [point for point in input_list if point not in p_t_r]

    return points_to_keep

# coords = [[12,24], [5, 12],[100,1020], [20,30], [121,214], [15,12]]

# print(itertools.combinations(data_subset[['x', 'y']][1:250].to_numpy(), 1))

# ptr = process(coords)

ptk = process(data_subset[['x', 'y']].to_numpy())

ptk_clean = np.vstack(ptr_2)

print(ptk_clean)

# print(data_subset[['x', 'y']].to_numpy()[52])

# print(coords[0])
# len(ptr)

# >>> [[12, 24], [100, 1020], [121, 214]]
